In [25]:
import json
import requests


In [26]:
with open('nodes.json', 'r') as f:
    nodes = json.load(f)

In [27]:
# get the data from OM2M
def getData(url):
    try:
        headers = {
        'X-M2M-Origin': 'iiith_guest:iiith_guest',
        'Accept': 'application/json'
        }
        response = requests.request("GET", url, headers=headers)
        return response.json()
    except:
        return {
            "m2m:cin": None
        }


In [28]:
nodesdata = {}

In [32]:
for node in nodes:
    print('getting data for node: ', node)
    for sensor in nodes[node]:
        print('\tgetting data for sensor: ', sensor)
        if node not in ['AE-SR', 'AE-WM']:
            url = 'http://onem2m.iiit.ac.in:443/~/in-cse/in-name/' + node + '/' + sensor + '/Data/la'
        else:
            subnode = sensor.split('-')[0] + '-' + sensor.split('-')[1]
            url = 'http://onem2m.iiit.ac.in:443/~/in-cse/in-name/' + node + '/' + subnode + '/' + sensor + '/Data/la'
            print(url)
        response = getData(url)
        nodesdata[sensor] = response['m2m:cin']

getting data for node:  AE-AQ
	getting data for sensor:  AQ-MG00-00
	getting data for sensor:  AQ-BN00-00
	getting data for sensor:  AQ-KN00-00
	getting data for sensor:  AQ-SN00-00
	getting data for sensor:  AQ-VN90-00
	getting data for sensor:  AQ-KH00-00
	getting data for sensor:  AQ-AN00-00
	getting data for sensor:  AQ-PH03-00
	getting data for sensor:  AQ-PL00-00
	getting data for sensor:  AQ-FG00-00
	getting data for sensor:  AQ-AD95-00
	getting data for sensor:  AQ-MG00-00
	getting data for sensor:  AQ-BN00-00
	getting data for sensor:  AQ-KN00-00
	getting data for sensor:  AQ-SN00-00
	getting data for sensor:  AQ-VN90-00
	getting data for sensor:  AQ-KH00-00
	getting data for sensor:  AQ-AN00-00
	getting data for sensor:  AQ-PH03-00
	getting data for sensor:  AQ-PL00-00
	getting data for sensor:  AQ-FG00-00
	getting data for sensor:  AQ-AD95-00
getting data for node:  AE-CM
	getting data for sensor:  CM-MG00-00
	getting data for sensor:  CM-VN90-00
	getting data for sensor:  C

In [33]:
with open('nodesdata.json', 'w') as f:
    json.dump(nodesdata, f, indent=4)

In [35]:
# Count all the nodes which are null
count = 0
nullnodes = []
for node in nodesdata:
    if nodesdata[node] is None:
        count += 1
        nullnodes.append(node)

In [40]:
samenodes = {}
# for each nullnode find closest node which has data
for nullnode in nullnodes:
    # nullnode is xxx-xxx-xxx-xx and each node is xxx-xxx-xxx-xx
    # try to match from end, first search for exact match
    # if null search till last but one - i.e. xxx-xxx-xxx
    # if still search for last but two - i.e. xxx-xxx
    samenode = ''
    for node in nodesdata:
        if nodesdata[node] is not None:
            if '-'.join(node.split('-')[:-1]) in nullnode:
                samenode = node
                break
    if samenode == '':
        if nodesdata[node] is not None:
            for node in nodesdata:
                if '-'.join(node.split('-')[:-2]) in nullnode:
                    samenode = node
                    break  
    if samenode == '':
        if nodesdata[node] is not None:
            for node in nodesdata:
                if '-'.join(node.split('-')[:-3]) in nullnode:
                    samenode = node
                    break
    samenodes[nullnode] = samenode

In [42]:
# poppulate nullnodes with data from samenodes
for nullnode in samenodes:
    nodesdata[nullnode] = nodesdata[samenodes[nullnode]]

In [43]:
with open('nodesdata.json', 'w') as f:
    json.dump(nodesdata, f, indent=4)